# 비트코인 단기 가격 예측 가능성 검증 실험 개요

## 1) 실험 목적
- 단기(다음 1분) 가격방향(상승/하락)이 과거 1분~30분의 정보로 유의미하게 예측 가능한지 통계·모형·경제성 관점에서 검증한다.

## 2) 가설
- 귀무가설(H0): 과거 정보로부터 다음 1분의 방향은 무작위(예측 불가능)하다.  
- 대립가설(H1): 일정한 특징(feature)을 사용하면 다음 1분 방향을 랜덤보다 유의미하게 잘 예측할 수 있다.

## 3) 데이터
- 입력: 1분 OHLCV 시계열 (timestamp, open, high, low, close, volume).

## 4) 라벨 정의
- 기본 라벨: `y_t = 1 if close_{t+1} > close_t else 0`.  
- 옵션: 노이즈 제거용 임계값 사용 (`y_t = 1 if return > δ`, δ 예: 0.0001(0.01%)).  
- 라벨 생성 후 라벨 비율(상승:하락)을 확인하여 불균형 여부 파악.

## 5) 피쳐 설계 — 기본 세트
- 과거 수익률 랙: `ret_lag_1`, `ret_lag_3`, `ret_lag_5`, `ret_lag_10`, `ret_lag_30`  
- 과거 거래량 랙: `vol_lag_1`, `vol_lag_3`, `vol_lag_5`, `vol_lag_10`, `vol_lag_30` (변화율 포함)  
- 이동평균 기반: `EMA(10)`, `EMA(30)`, `ema_diff = (EMA10 - EMA30)/close`  
- 변동성/ATR: rolling std(10), ATR(14)  
- 캔들 비율: `(close-open)/range`, `high-low` 등  
- 추가(옵션): VWAP, OBV, 최근 n분의 상승빈도  
- 표준화: 필요시 로그 변환 또는 StandardScaler

## 6) 데이터 분할
- 고민중

## 7) 모델
- 베이스라인: Logistic Regression  
- 비선형 모델: RandomForest, LightGBM, XGBoost  
- 비교: 단순 룰(문턱 기반) 및 무작위 분류기  
- 하이퍼파라미터: 시계열 CV 기반 그리드/랜덤 서치

## 8) 평가 지표
- 분류 성능: Accuracy, Precision, Recall, F1, ROC-AUC  
- 확률품질: Brier score, calibration plot  
- 경제성: 평균 단위거래 수익, 누적수익, 연환산 샤프, 거래비용·슬리피지 민감도  
- 통계검정: 퍼뮤테이션(라벨 셔플링) 기반 p-value (기준 p < 0.05)

## 9) 통계적 유의성 절차
1. 모델 학습 및 테스트 AUC 계산.  
2. 라벨을 랜덤으로 n_perm번(예: 500) 셔플하여 동일한 절차로 AUC 분포 생성.  
3. 원본 AUC가 셔플 분포에서 상위 몇 %인지로 p-value 산출(한쪽 검정).  
4. 부트스트랩(예: 1000 샘플)으로 AUC 신뢰구간(CI) 추정.

## 10) 경제성(백테스트) 절차
- 신호 생성: `prob > thresh` → 다음 1분 long 진입(보유 1분). thresh 예: 0.55, 0.6  
- 거래비용: per-trade cost 0.0005(0.05%) 기본; 0.02%~0.2% 민감도  
- 성과지표: 누적수익, MDD, 거래비용 반영 후 샤프  
- 슬리피지/실행 제약 고려(마켓 임팩트는 별도)

## 11) 견고성/추가 분석
- 시간 프레임 민감도: 1분 → 5분 → 15분  
- 라벨 임계(δ) 민감도 분석  
- 특징 중요도: SHAP 또는 feature importance  
- 레짐별 성능: 변동성 레짐(낮음/중간/높음) 비교

## 12) 성공 기준(Decision rules)
- 통계적: 테스트 AUC > 0.5, permutation p < 0.05  
- 경제적: 거래비용 반영 후 연환산 샤프 > 0.5, buy&hold 대비 우위 또는 낮은 MDD  
- 실무적: 시계열 CV에서 일관된 성능(과적합 아님)

## 13) 산출물(Deliverables)
1. 실험 노트(데이터 범위, 전처리, 파라미터)  
2. 모델 성능 요약표(AUC, Accuracy, p-value, CI)  


## 14) 위험/한계
- 데이터 누수 및 타임스탬프 오류 주의  
- 고빈도 시계열의 표본의존성·비정상성 가능  


In [1]:
#data processing
import kagglehub
import pandas as pd
import os
path = kagglehub.dataset_download("mczielinski/bitcoin-historical-data")
files = os.listdir(path)
data = pd.read_csv(f"{path}/{files[0]}")
data.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.set_index('timestamp', drop = False)
data.head()

,timestamp,open,high,low,close,volume
timestamp,,,,,,
2012-01-01 10:01:00,2012-01-01 10:01:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:02:00,2012-01-01 10:02:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:03:00,2012-01-01 10:03:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:04:00,2012-01-01 10:04:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:05:00,2012-01-01 10:05:00,4.58,4.58,4.58,4.58,0.0
